# Imports

In [21]:
import os
import sys

import numpy as np
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt
import cv2

from src.sam_segmentation.yolo import load_yolov8_detector, yolov8_detect, draw_bounding_boxes
from src.sam_segmentation.sam import load_sam_predictor, sam_segmentation
from src.sam_segmentation.utils import unite_masks

# Data

In [26]:
DATA_DIR = r"C:\Internship\ITMO_ML\data\Frames\Bubbles_every_frame"

In [27]:
folders_list = os.listdir(DATA_DIR)
folders_list

['F1_1_1_1_frames',
 'F1_1_1_2_frames',
 'F1_1_2_1_frames',
 'F1_1_2_2_frames',
 'F1_1_3_1_frames',
 'F1_1_3_2_frames',
 'F1_1_4_1_frames',
 'F1_1_4_2_frames',
 'F1_1_5_1_frames',
 'F1_1_5_2_frames',
 'F1_2_2_1_frames',
 'F1_2_2_2_frames',
 'F1_2_3_1_frames',
 'F1_2_3_2_frames']

In [28]:
def count_images(image_folder_path):
    images_num = len(os.listdir(image_folder_path))
    return images_num

def summarize_folders(data_dir, folders_list):
    folders_info = {}
    images_sum = 0
    
    for folder in folders_list:
        images_num = count_images(os.path.join(data_dir, folder))
        folders_info[folder] = images_num
        images_sum += images_num
        
    print(f"There is {images_sum} images:")
    for folder in folders_info.keys():
        print(f"\t{folder} folder has {folders_info[folder]} images")
    

In [29]:
summarize_folders(DATA_DIR, folders_list)

There is 20721 images:
	F1_1_1_1_frames folder has 1787 images
	F1_1_1_2_frames folder has 1798 images
	F1_1_2_1_frames folder has 1508 images
	F1_1_2_2_frames folder has 1491 images
	F1_1_3_1_frames folder has 1479 images
	F1_1_3_2_frames folder has 1512 images
	F1_1_4_1_frames folder has 1797 images
	F1_1_4_2_frames folder has 1783 images
	F1_1_5_1_frames folder has 1502 images
	F1_1_5_2_frames folder has 1493 images
	F1_2_2_1_frames folder has 1474 images
	F1_2_2_2_frames folder has 1484 images
	F1_2_3_1_frames folder has 1480 images
	F1_2_3_2_frames folder has 133 images


# SAM+YOLO Labeling

Yolo detector:

In [30]:
custom_yolo_checkpoint_path = r"C:\Internship\ITMO_ML\CTCI\checkpoints\yolov8\yolov8s_on_bubbles_24\run3\weights\best.pt"
detector = load_yolov8_detector(custom_yolo_checkpoint_path)

SAM:

In [31]:
sam_checkpoint = r"C:\Internship\ITMO_ML\CTCI\checkpoints\sam_checkpoints\sam_vit_h_4b8939.pth"
model_type = "vit_h"
device="cuda"

predictor = load_sam_predictor(checkpoint_path=sam_checkpoint, model_type=model_type, device=device)

## Labeling

In [32]:
def segment_images_from_folder(folder):
    masks_folder_name = folder + "_masks"
    dirlist = os.listdir(DATA_DIR)
    if masks_folder_name not in dirlist:
        os.mkdir(os.path.join(DATA_DIR, masks_folder_name))
    
    images_list = os.listdir(os.path.join(DATA_DIR, folder))
    masks_list = os.listdir(os.path.join(DATA_DIR, masks_folder_name))
    for image_name in tqdm(images_list):
        if image_name in masks_list:
            continue
        image = cv2.imread(os.path.join(DATA_DIR, folder,image_name))
        boxes = yolov8_detect(image=image, detector=detector,    return_objects=False)
        masks_list = sam_segmentation(image=image,predictor=predictor,  boxes=boxes, prompt_point=True)
    
        masks_united = []
        for masks in masks_list:
            masks_united.append(unite_masks(masks))
        mask = unite_masks(masks_united)
        cv2.imwrite(filename=os.path.join(DATA_DIR,masks_folder_name,   image_name), img=mask)

    print(f"Images from the folder {folder} were segmented!")

In [ ]:
folder = folders_list[1]
segment_images_from_folder(folder)

  0%|          | 0/1798 [00:00<?, ?it/s]

0: 608x800 300 Bubbless, 58.8ms
Speed: 8.0ms preprocess, 58.8ms inference, 4.0ms postprocess per image at shape (1, 3, 608, 800)


  0%|          | 1/1798 [00:56<28:12:20, 56.51s/it]

0: 608x800 300 Bubbless, 79.8ms
Speed: 3.9ms preprocess, 79.8ms inference, 6.0ms postprocess per image at shape (1, 3, 608, 800)


C:\Users\PC\AppData\Local\Temp\ipykernel_8556\11334664.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if image_name in masks_list:
  0%|          | 2/1798 [01:50<27:35:54, 55.32s/it]

...

0: 608x800 300 Bubbless, 54.5ms
Speed: 5.0ms preprocess, 54.5ms inference, 5.1ms postprocess per image at shape (1, 3, 608, 800)


 31%|███       | 555/1798 [8:45:17<18:28:00, 53.48s/it]

0: 608x800 300 Bubbless, 100.8ms
Speed: 4.0ms preprocess, 100.8ms inference, 4.0ms postprocess per image at shape (1, 3, 608, 800)


 31%|███       | 556/1798 [8:46:09<18:20:07, 53.15s/it]

0: 608x800 300 Bubbless, 87.8ms
Speed: 12.0ms preprocess, 87.8ms inference, 8.0ms postprocess per image at shape (1, 3, 608, 800)
